#Assignment 7

Grupo 5  :Karla Chauca, Alejandra Navarro, Adriana Sierra y  Elisa Vivar

In [ ]:
#pip install openpyxl

In [ ]:
import openpyxl

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import chardet
from shapely.ops import unary_union

In [ ]:
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import folium 
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap

In [ ]:
# Gettting the character format (encoding type)

base = open(r'../../_data/data_dengue_peru.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']
charenc

#Casos de Dengue

In [ ]:
cv_data = pd.read_csv( r'../../_data/data_dengue_peru.csv', encoding = charenc)
cv_data.head()

In [ ]:
# Limpiar los valores eliminando los separadores de miles
cv_data['Casos'] = cv_data['Casos'].astype(str).str.replace(',', '', regex=False)

# Convertir la columna a valores numéricos, manteniendo NA
cv_data['Casos'] = pd.to_numeric(cv_data['Casos'], errors='coerce')

In [ ]:
# cv_data['Casos'].isna().sum()

In [ ]:
# cv_data['Ubigeo'].dtype

In [ ]:
# Función para convertir a str y agregar un 0 si la longitud es 5
def format_ubigeo(ubigeo):
    if pd.isna(ubigeo):
        return ubigeo  # Mantener NaN como está
    ubigeo_str = str(ubigeo)
    if len(ubigeo_str) == 5:
        return '0' + ubigeo_str
    return ubigeo_str

# Aplicar la función a la columna 'Ubigeo'
cv_data['Ubigeo'] = cv_data['Ubigeo'].apply(format_ubigeo)

cv_data['ubigeo_dep'] = cv_data['Ubigeo'].str[:2]
cv_data['ubigeo_prov'] = cv_data['Ubigeo'].str[:4]

cv_data = cv_data.rename({'Ubigeo':'ubigeo'}, axis =1 )
cv_data #así queda el df de casos de dengue

Shapefile

In [ ]:
# Upload shape file at district level
maps = gpd.read_file(r'../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp')

In [ ]:
maps = maps.rename({'UBIGEO':'ubigeo'}, axis =1 )

maps['ubigeo_dep'] = maps['ubigeo'].str[:2]
maps['ubigeo_prov'] = maps['ubigeo'].str[:4]
maps = maps[['ubigeo','ubigeo_dep','ubigeo_prov','geometry']]
print(maps.crs)

In [ ]:
# Agrupar por provincia
gdf_prov = maps.groupby('ubigeo_prov').agg({
    'geometry': lambda x: unary_union(x) 
}).reset_index()

# Convertir a GeoDataFrame
gdf_prov = gpd.GeoDataFrame(gdf_prov, geometry='geometry')

# Asegurarse de que el CRS está definido
gdf_prov.set_crs(maps.crs, inplace=True)

# Verificar el GeoDataFrame resultante
print(gdf_prov.crs)

# Shapefile de provincia: gdf_prov

In [ ]:
# Agrupar por departamento
gdf_dep = maps.groupby('ubigeo_dep').agg({
    'geometry': lambda x: unary_union(x) 
}).reset_index()

# Convertir a GeoDataFrame
gdf_dep = gpd.GeoDataFrame(gdf_dep, geometry='geometry')

# Asegurarse de que el CRS está definido
gdf_dep.set_crs(maps.crs, inplace=True)

# Verificar el GeoDataFrame resultante
print(gdf_dep.crs)

# Shapefile de provincia: gdf_dep

In [ ]:
gdf_dist=maps[['ubigeo','geometry']]
print(gdf_dist.crs)

# Shapefile de distrito: gdf_dist

In [ ]:
# Gráficos de mapas
fig, axs = plt.subplots(1, 3, figsize=(11, 5), sharex=True, sharey=True)

# Graficar el GeoDataFrame de distritos
gdf_dist.plot(ax=axs[0], edgecolor='lightgrey', color='black')
axs[2].set_title('Departamentos')

# Graficar el GeoDataFrame de provincias
gdf_prov.plot(ax=axs[1], edgecolor='lightgrey', color='black')
axs[1].set_title('Provincias')

# Graficar el GeoDataFrame de departamentos
gdf_dep.plot(ax=axs[2], edgecolor='lightgrey', color='black')
axs[0].set_title('Distritos')

# Ajustar el espaciado entre subgráficas
plt.tight_layout()

# Mostrar la figura
plt.show()

In [1]:
### Año 2021 - distrito

In [ ]:
grouped1 = cv_data.groupby(['ubigeo', 'Año']).agg({
    'Casos': 'sum', # Sumar los casos
    'Distrito': 'first'
}).reset_index()

df1 = grouped1[grouped1['Año'] == 2021]
df1 = df1.reset_index(drop=True)

In [ ]:
df_dist = pd.merge(gdf_dist, df1, how="outer", on=["ubigeo"])

df_dist = gpd.GeoDataFrame(df_dist, geometry='geometry')
df_dist.set_crs(epsg=4326, inplace=True)
print(df_dist.crs)

In [ ]:
# Máscara para valores faltantes
mask_na = df_dist['Casos'].isna()

fig, ax = plt.subplots(figsize=(10, 10))

# Graficar los valores no nulos con una leyenda continua
df_dist[~mask_na].plot(column='Casos', cmap='RdPu', edgecolor='gray', ax=ax, legend=True, linestyle='-')

# Graficar los valores NaN en color diferente
df_dist[mask_na].plot(edgecolor='white', ax=ax, color='lightgray', linestyle='-')

# Crear la leyenda para los valores NaN
handles, labels = ax.get_legend_handles_labels()
handles.append(plt.Line2D([0], [0], color='lightgray', linewidth=2, linestyle='-', label='No Datos'))
labels.append('Sin Datos')

# Configurar la leyenda continua
# Añadir la leyenda continua para los valores no nulos
ax.legend(handles=handles, labels=labels, title='Número de Casos', loc='lower left', bbox_to_anchor=(0, 0), fontsize='small')

# Agregar el título al mapa
ax.set_title("Número de casos, por distrito (2021)", fontsize=16)

plt.show()